# FitCoach Live Feedback - Webcam or Single Video Test

IMPORTANT NOTES: Make sure to update any file paths to match your repository.

It is recommended to run on an A-100 GPU.

---

## Step 1: Install Dependencies
Note: You may observe some errors related to pip's dependency resolver, which you can ignore.

Restart Runtime after installing and go directly to Step 2.

In [ ]:
# Core dependencies
print("[1/12] Installing PyYAML...")
!pip install -q PyYAML==6.0

print("[2/12] Installing datasets...")
!pip install -q datasets==2.14.6

print("[3/12] Installing evaluate...")
!pip install -q evaluate==0.4.1

print("[4/12] Installing OpenCV...")
!pip install -q opencv-python==4.9.0.80

print("[5/12] Installing transformers...")
!pip install -q transformers==4.36.0

print("[6/12] Installing accelerate...")
!pip install -q accelerate==0.24.1

print("[7/12] Installing peft...")
!pip install -q peft==0.5.0

print("[8/12] Installing bitsandbytes...")
!pip install -q bitsandbytes>=0.44.0

print("[9/12] Installing tqdm...")
!pip install -q tqdm

print("[10/12] Installing rouge_score...")
!pip install -q rouge_score

print("[11/12] Installing bert_score...")
!pip install -q bert_score

# Fix NumPy compatibility (OpenCV requires NumPy 1.x)
print("[12/12] Fixing NumPy compatibility...")
!pip install -q "numpy<2"

print("\n" + "="*60)
print("Setup complete! All required dependencies installed.")
print("="*60)
print("\nIMPORTANT: Restart Runtime")
print("After restart, go directly to Step 2")

[1/12] Installing PyYAML...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
[2/12] Installing datasets...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 

## Step 2: Mount Google Drive and Setup Paths

IMPORTANT NOTES: Make sure to update any file paths to match your repository.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to shared drive
%cd /content/drive/Shareddrives/'CIS6800 final project'

import os
DRIVE_ROOT = os.getcwd()
DOWNLOADS_DIR = os.path.join(DRIVE_ROOT, "downloads")

# Create downloads directory
os.makedirs(DOWNLOADS_DIR, exist_ok=True)

print(f"Drive root: {DRIVE_ROOT}")
print(f"Downloads dir: {DOWNLOADS_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/CIS6800 final project
Drive root: /content/drive/Shareddrives/CIS6800 final project
Downloads dir: /content/drive/Shareddrives/CIS6800 final project/downloads


## Step 2: Clone Repository

In [ ]:
# Navigate to /content for repo
%cd /content

# Remove if exists
import shutil
if os.path.exists('/content/FitCoach'):
    print("Removing existing FitCoach directory...")
    shutil.rmtree('/content/FitCoach')

print("Cloning repository...\n")
!git clone -b live-feedback https://github.com/bryanaalfaro/FitCoach.git
%cd FitCoach
print("\nRepository cloned!")

/content
Cloning repository...

Cloning into 'FitCoach'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 139 (delta 24), reused 19 (delta 19), pack-reused 103 (from 1)
Receiving objects: 100% (139/139), 926.68 KiB | 37.07 MiB/s, done.
Resolving deltas: 100% (58/58), done.
/content/FitCoach

Repository cloned!


## Step 4: Download Models

### 4a: Login to HuggingFace

**Required:** Get access to LLaMA-2
1. Go to: https://huggingface.co/meta-llama/Llama-2-7b-hf
2. Click "Request access"
3. Get token: https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import notebook_login

print("Please login with your HuggingFace token:")
print("Get token: https://huggingface.co/settings/tokens\n")

notebook_login()

Please login with your HuggingFace token:
Get token: https://huggingface.co/settings/tokens



### 4b: Download LLaMA-2-7B

**Cached in Google Drive** - Downloads only once

In [ ]:
from huggingface_hub import snapshot_download
import os

# Re-define paths in case runtime was restarted
DRIVE_ROOT = "/content/drive/Shareddrives/CIS6800 final project"
DOWNLOADS_DIR = os.path.join(DRIVE_ROOT, "downloads")
LLAMA_DIR = os.path.join(DOWNLOADS_DIR, "models/Llama-2-7b-hf")

# # Check if already downloaded - verify actual model files exist
# model_file = os.path.join(LLAMA_DIR, "model-00001-of-00002.safetensors")
# config_file = os.path.join(LLAMA_DIR, "config.json")

# if os.path.exists(config_file) and os.path.exists(model_file):
#     # Check file size to ensure it's complete (~10GB)
#     model_size = os.path.getsize(model_file) / (1024**3)  # Size in GB
#     if model_size > 9.5:  # Should be ~9.98GB
#         print("LLaMA-2-7B already downloaded in Drive")
#         print(f"Location: {LLAMA_DIR}")
#     else:
#         print(f"Partial download detected ({model_size:.2f}GB / 9.98GB)")
#         print("Resuming download...\n")
#         download_needed = True
# else:
#     print("Downloading LLaMA-2-7B (~13GB)...")
#     print("This will take 15-25 minutes...")
#     print("(If download stalls, just stop and re-run this cell - it will resume)\n")
#     download_needed = True

# if 'download_needed' in locals() and download_needed:
#     os.makedirs(LLAMA_DIR, exist_ok=True)

#     # Download with resume capability
#     try:
#         snapshot_download(
#             repo_id="meta-llama/Llama-2-7b-hf",
#             local_dir=LLAMA_DIR,
#             local_dir_use_symlinks=False,
#             resume_download=True,
#             max_workers=4  # Limit parallel downloads to reduce stalling
#         )
#         print("\nLLaMA-2-7B downloaded to Drive!")
#     except Exception as e:
#         print(f"\nDownload interrupted: {e}")
#         print("Re-run this cell to resume download")
#         raise

# Create symlink in FitCoach directory
%cd /content/FitCoach
if os.path.exists("./Llama-2-7b-hf"):
    os.remove("./Llama-2-7b-hf")
os.symlink(LLAMA_DIR, "./Llama-2-7b-hf")
print(f"\nSymlinked to /content/FitCoach/Llama-2-7b-hf")

/content/FitCoach

Symlinked to /content/FitCoach/Llama-2-7b-hf


### 4c: Download 3D CNN weights

In [ ]:
import os

CNN_DIR = os.path.join(DOWNLOADS_DIR, "models/ckpts_efficientnet")
CNN_WEIGHTS = os.path.join(CNN_DIR, "fitness_ally_hypermodel/efficientnet4Lite_1.8.3.checkpoint")

if os.path.exists(CNN_WEIGHTS):
    print("3D CNN weights already downloaded in Drive")
    print(f"Location: {CNN_DIR}")
else:
    print("Downloading 3D CNN weights...")
    print("(wget auto-resumes if interrupted - just re-run this cell)\n")
    os.makedirs(CNN_DIR, exist_ok=True)

    # Use Python to download instead of wget to avoid path issues
    import urllib.request
    import shutil

    url = "https://github.com/Qualcomm-AI-research/FitCoach/releases/download/v1.0/efficientnet_3d_cnn_weights.tar.gz"
    tar_file = os.path.join(CNN_DIR, "efficientnet_3d_cnn_weights.tar.gz")

    print("Downloading...")
    urllib.request.urlretrieve(url, tar_file)

    print("\nExtracting (note: file is .tar not .tar.gz despite name)...")
    import tarfile
    with tarfile.open(tar_file, 'r') as tar:
        tar.extractall(path=CNN_DIR)

    print("3D CNN weights downloaded to Drive!")

# Create symlink
%cd /content/FitCoach
if os.path.exists("./ckpts_efficientnet"):
    os.remove("./ckpts_efficientnet")
os.symlink(CNN_DIR, "./ckpts_efficientnet")
print(f"\nSymlinked to /content/FitCoach/ckpts_efficientnet")

3D CNN weights already downloaded in Drive
Location: /content/drive/Shareddrives/CIS6800 final project/downloads/models/ckpts_efficientnet
/content/FitCoach

Symlinked to /content/FitCoach/ckpts_efficientnet


### 4d: Download Stream-VLM weights

In [ ]:
import os

STREAMVLM_DIR = os.path.join(DOWNLOADS_DIR, "models/ckpts_streamvlm")
STREAMVLM_WEIGHTS = os.path.join(STREAMVLM_DIR, "ckpts_streamvlm/state_dict.pth.tar")

if os.path.exists(STREAMVLM_WEIGHTS):
    print("Stream-VLM weights already downloaded in Drive")
    print(f"Location: {STREAMVLM_DIR}")
else:
    print("Downloading Stream-VLM weights (6 parts, ~3.5GB total)...")
    print("This will take 5-10 minutes...")
    print("(If interrupted, just re-run this cell)\n")
    os.makedirs(STREAMVLM_DIR, exist_ok=True)

    # Use Python to download to avoid path issues with wget
    import urllib.request

    parts = ['aa', 'ab', 'ac', 'ad', 'ae', 'af']
    base_url = "https://github.com/Qualcomm-AI-research/FitCoach/releases/download/v1.0/streamvlm_weights.tar.gz."

    # Download each part
    for i, part in enumerate(parts, 1):
        print(f"Downloading part {i}/6...")
        url = base_url + part
        dest = os.path.join(STREAMVLM_DIR, f"streamvlm_weights.tar.gz.{part}")
        urllib.request.urlretrieve(url, dest)

    print("\nExtracting...")
    import subprocess

    # Combine and extract
    part_files = [os.path.join(STREAMVLM_DIR, f"streamvlm_weights.tar.gz.{p}") for p in parts]
    combined = os.path.join(STREAMVLM_DIR, "streamvlm_weights.tar.gz")

    # Concatenate parts
    with open(combined, 'wb') as outfile:
        for part_file in part_files:
            with open(part_file, 'rb') as infile:
                outfile.write(infile.read())

    # Extract
    import tarfile
    with tarfile.open(combined, 'r:gz') as tar:
        tar.extractall(path=STREAMVLM_DIR)

    print("Stream-VLM weights downloaded to Drive!")

# Create symlink
%cd /content/FitCoach
if os.path.exists("./ckpts_streamvlm"):
    os.remove("./ckpts_streamvlm")
os.symlink(STREAMVLM_DIR, "./ckpts_streamvlm")
print(f"\nSymlinked to /content/FitCoach/ckpts_streamvlm")

print("\n" + "="*60)
print("All models downloaded! Ready for evaluation.")
print("="*60)

Stream-VLM weights already downloaded in Drive
Location: /content/drive/Shareddrives/CIS6800 final project/downloads/models/ckpts_streamvlm
/content/FitCoach

Symlinked to /content/FitCoach/ckpts_streamvlm

All models downloaded! Ready for evaluation.


In [ ]:
import sys
sys.path.insert(0, '/content/FitCoach')

import yaml
import torch
from src.model_helpers import make_model
from scripts.live_feedback_lightweight import LightweightFeedbackCoach

%cd /content/FitCoach

# Use lightweight config for evaluation
config_path = "/content/FitCoach/configs/live_lightweight.yaml"
print(f"Using config: {config_path}")

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("\nConfig settings:")
print(f"  Feedback interval: {config['evaluator']['sampling_kwargs']['feedback_interval']}s")
print(f"  Feature frequency: {config['evaluator']['sampling_kwargs']['feats_frequency']} fps")
print(f"  Max feedback length: {config['evaluator']['sampling_kwargs']['max_feedback_length']}")

# Load model
print("\nLoading Stream-VLM model...")
llama2_7b_path = config["model"]["llama2_7b_path"]
model_kwargs = config["model"]["kwargs"]
stream_vlm = make_model(llama2_7b_path, **model_kwargs)
stream_vlm.eval()
print("Model loaded")

# Initialize coach
cnn_weights_path = "./ckpts_efficientnet/fitness_ally_hypermodel/efficientnet4Lite_1.8.3.checkpoint"
coach = LightweightFeedbackCoach(
    model=stream_vlm,
    config=config,
    cnn_weights_path=cnn_weights_path,
    max_buffer_size=200
)
print("Coach initialized")

print("\n" + "="*60)
print("Ready for evaluation!")
print("="*60)

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


/content/FitCoach
Using config: /content/FitCoach/configs/live_lightweight.yaml

Config settings:
  Feedback interval: 15.0s
  Feature frequency: 2 fps
  Max feedback length: 48

Loading Stream-VLM model...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'str' object has no attribute 'model' <class 'function'>  cannot be sent to  cuda
Model loaded
Loading 3D CNN for feature extraction...
3D CNN loaded successfully!
Coach initialized

Ready for evaluation!


## Step 5: Choose Video Source

**Option A: Upload a video file** - Run the cell below to upload

**Option B: Live webcam feedback** - Skip to Option B section below

In [ ]:
from google.colab import files

print("Please upload your workout video:")
print("Supported: MP4, AVI, MOV")
print("Recommended: 30-120 seconds\n")

uploaded = files.upload()

if uploaded:
    video_filename = list(uploaded.keys())[0]
    use_webcam = False
    print(f"\nVideo uploaded: {video_filename}")
else:
    print("\nNo video uploaded")
    video_filename = None
    use_webcam = False

Please upload your workout video:
Supported: MP4, AVI, MOV
Recommended: 30-120 seconds



Saving SquatsTrimmed2.mp4 to SquatsTrimmed2.mp4

Video uploaded: SquatsTrimmed2.mp4


### Option B: Live Webcam Feedback

Run the cell below to use your webcam for real-time coaching:
- Webcam streams continuously
- Model gives feedback every 5-15 seconds depending on config
- See yourself with feedback overlay
- Press Stop button to end

In [ ]:
# ===== CONFIGURE HERE =====
exercise_type = "squats"   # "push-ups", "jumping-jacks", etc.
display_every_feedback = True   # print feedback immediately
# ==========================

import os, sys, time
import numpy as np
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
from io import BytesIO

# Ensure repo root for relative paths (important)
os.chdir("/content/FitCoach")
sys.path.insert(0, "/content/FitCoach")

# ---- Require that config/coach are already loaded ----
assert "config" in globals(), "config not found. Run the model/config cell first."
assert "coach" in globals(), "coach not found. Run the model/config cell first."

# Reset coach state (matches evaluator intent)
coach.feature_buffer.clear()
coach.feedback_history.clear()

# ---------------- JS helpers ----------------
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    return np.array(Image.open(BytesIO(image_bytes)))

def start_webcam():
    js = Javascript(r'''
        async function startWebcam() {
            // Remove any previous webcam containers
            const old = document.getElementById("fitcoach-webcam-root");
            if (old) old.remove();

            const root = document.createElement('div');
            root.id = "fitcoach-webcam-root";
            document.body.appendChild(root);

            const video = document.createElement('video');
            video.id = "fitcoach-video";
            video.style.display = 'block';
            video.width = 640;
            video.height = 480;

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();
            root.appendChild(video);

            const feedbackDiv = document.createElement('div');
            feedbackDiv.id = 'feedback';
            feedbackDiv.style.position = 'fixed';
            feedbackDiv.style.bottom = '20px';
            feedbackDiv.style.left = '20px';
            feedbackDiv.style.padding = '10px';
            feedbackDiv.style.background = 'rgba(0, 100, 0, 0.85)';
            feedbackDiv.style.color = 'white';
            feedbackDiv.style.fontSize = '18px';
            feedbackDiv.style.zIndex = '9999';
            feedbackDiv.style.maxWidth = '70vw';
            feedbackDiv.style.whiteSpace = 'pre-wrap';
            feedbackDiv.textContent = 'Starting...';
            root.appendChild(feedbackDiv);

            return true;
        }
    ''')
    display(js)
    return eval_js('startWebcam()')

def capture_frame():
    js = Javascript(r'''
        async function captureFrame() {
            const video = document.getElementById('fitcoach-video');
            if (!video) return null;

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth || 640;
            canvas.height = video.videoHeight || 480;
            canvas.getContext('2d').drawImage(video, 0, 0);

            return canvas.toDataURL('image/jpeg', 0.8);
        }
    ''')
    display(js)
    return eval_js('captureFrame()')

def update_feedback(text):
    # Escape backticks and backslashes for JS template literal safety
    safe = text.replace("\\", "\\\\").replace("`", "\\`")
    js = Javascript(f'''
        const elem = document.getElementById('feedback');
        if (elem) elem.textContent = `Coach: {safe}`;
    ''')
    display(js)

# ---------------- Start webcam ----------------
print("Starting webcam...")
start_webcam()
time.sleep(1.5)

system_prompt = (
    "You are an expert fitness coaching AI who coaches users as they exercise. "
    "You assess their performance, count repetitions, and proactively provide feedback. "
    f"The user should be doing {exercise_type}."
)

# Timing params from config (same fields you printed earlier)
feedback_interval = float(config["evaluator"]["sampling_kwargs"]["feedback_interval"])
feats_frequency = float(config["evaluator"]["sampling_kwargs"]["feats_frequency"])
feature_interval = 1.0 / feats_frequency

print(f"\nLive feedback for {exercise_type} starting...")
print(f"Feature frequency: {feats_frequency} fps (every {feature_interval:.2f}s)")
print(f"Feedback interval: {feedback_interval:.1f}s")
print("Use the notebook Stop button to end.\n")

last_feedback_time = time.time()
last_feature_time = time.time()
frame_count = 0
feedback_count = 0
oom_count = 0

try:
    while True:
        js_reply = capture_frame()
        if not js_reply:
            print("No frame received; stopping.")
            break

        frame = js_to_image(js_reply)
        frame_count += 1
        now = time.time()

        # ---- Feature extraction at feats_frequency ----
        if now - last_feature_time >= feature_interval:
            try:
                preprocessed = coach.preprocess_frame(frame)
                coach.feature_buffer.append(preprocessed)
                last_feature_time = now
            except Exception as e:
                # keep going; webcam is noisy
                print(f"Preprocessing error @ frame {frame_count}: {e}", flush=True)

        # ---- Feedback generation every feedback_interval ----
        if now - last_feedback_time >= feedback_interval:
            try:
                feedback, _ = coach.generate_feedback(system_prompt)
                last_feedback_time = now

                if feedback and feedback.strip():
                    feedback_count += 1
                    # Live print (doesn't get swallowed by buffering)
                    if display_every_feedback:
                        print(f"\n[{feedback_count:02d}] Coach: {feedback}", flush=True)
                    update_feedback(feedback)

            except torch.cuda.OutOfMemoryError:
                oom_count += 1
                try:
                    import torch
                    torch.cuda.empty_cache()
                except Exception:
                    pass
                # keep buffer bounded (same spirit as evaluator)
                while len(coach.feature_buffer) > 20:
                    coach.feature_buffer.popleft()
                print("\n[OOM] Cleared cache and trimmed buffer; continuing...", flush=True)
            except Exception as e:
                print(f"\nFeedback error @ frame {frame_count}: {e}", flush=True)
                last_feedback_time = now

        # ~30fps loop (capture is already slow in Colab; this is fine)
        time.sleep(0.03)

except KeyboardInterrupt:
    print("\nStopped by user")

print("\nSession complete!")
print(f"Total frames: {frame_count}")
print(f"Total feedback messages: {feedback_count}")
print(f"GPU OOM events: {oom_count}")

if getattr(coach, "feedback_history", None):
    print("\nFeedback history:")
    # Depending on implementation, feedback_history may be list[str] or list[(ts, str)]
    for i, item in enumerate(coach.feedback_history, 1):
        if isinstance(item, (tuple, list)) and len(item) == 2:
            ts, fb = item
            print(f"{i:02d}. {fb}")
        else:
            print(f"{i:02d}. {item}")



Starting webcam...


<IPython.core.display.Javascript object>


Live feedback for squats starting...
Feature frequency: 2.0 fps (every 0.50s)
Feedback interval: 15.0s
Use the notebook Stop button to end.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Stopped by user

Session complete!
Total frames: 83
Total feedback messages: 2
GPU OOM events: 0


## Step 6: Process Uploaded Video

**Only run this if you uploaded a video in Option A**

Skip this step if you used Option B (webcam)

In [ ]:
# ===== CONFIGURE HERE =====
exercise_type = "squats"          # e.g., "push-ups", "jumping-jacks"
max_minutes = None               # e.g., 2 to stop after 2 minutes; None = full video
# ==========================

import os, time
import cv2
import torch
from tqdm import tqdm

# Make sure we're running from the repo root (paths in config assume this)
os.chdir("/content/FitCoach")

if video_filename is None:
    print("No video uploaded. Either upload one in Option A or use Option B (webcam).")
else:
    system_prompt = (
        "You are an expert fitness coaching AI who coaches users as they exercise. "
        "You assess their performance, count repetitions, and proactively provide feedback. "
        f"The user should be doing {exercise_type}."
    )

    # ---- Pull timing params from the config (same as evaluator) ----
    feedback_interval = config["evaluator"]["sampling_kwargs"]["feedback_interval"]
    feats_frequency = config["evaluator"]["sampling_kwargs"]["feats_frequency"]
    feature_interval = 1.0 / feats_frequency

    print("Starting FitCoach Live Feedback...\n")
    print(f"Video: {video_filename}")
    print(f"Exercise: {exercise_type}")
    print(f"Feedback interval: {feedback_interval}s")
    print(f"Feature frequency: {feats_frequency} fps (every {feature_interval:.2f}s)")
    print("\n" + "="*60 + "\n")

    # ---- Reset coach state (matches evaluator) ----
    coach.feature_buffer.clear()
    coach.feedback_history.clear()

    # ---- Open video ----
    cap = cv2.VideoCapture(video_filename)
    if not cap.isOpened():
        raise RuntimeError(f"Error: Could not open video {video_filename}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps if fps and fps > 0 else 0.0

    # Optional early stop
    if max_minutes is not None:
        max_seconds = 60.0 * float(max_minutes)
        max_frames = int(max_seconds * fps) if fps and fps > 0 else None
        if max_frames is not None:
            total_frames = min(total_frames, max_frames)

    print(f"Video info: {total_frames} frames, {fps:.2f} fps, {duration:.1f}s")

    predictions = []
    pred_timestamps = []
    frame_count = 0
    oom_count = 0
    start_time = time.time()

    # Frame-based elapsed time tracking (same as evaluator)
    last_feedback_elapsed = -999.0
    last_feature_elapsed = -999.0

    with tqdm(total=total_frames, desc="Processing") as pbar:
        while frame_count < total_frames:
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            pbar.update(1)

            elapsed_time = (frame_count - 1) / fps if fps and fps > 0 else 0.0

            # ---- Feature extraction at feats_frequency (same pattern as evaluator) ----
            if elapsed_time - last_feature_elapsed >= feature_interval:
                try:
                    preprocessed = coach.preprocess_frame(frame)
                    coach.feature_buffer.append(preprocessed)
                    last_feature_elapsed = elapsed_time
                except torch.cuda.OutOfMemoryError:
                    oom_count += 1
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    # Keep buffer from growing too large after OOM (matches evaluator)
                    while len(coach.feature_buffer) > 20:
                        coach.feature_buffer.popleft()
                except Exception as e:
                    print(f"\nError at frame {frame_count}: {e}")

            # ---- Feedback generation every feedback_interval ----
            if elapsed_time - last_feedback_elapsed >= feedback_interval:
                try:
                    feedback, _ = coach.generate_feedback(system_prompt)
                    if feedback and feedback.strip():
                        predictions.append(feedback)
                        pred_timestamps.append(elapsed_time)

                        # 🔴 LIVE PRINT (immediate)
                        print(f"\n[{elapsed_time:6.1f}s] FEEDBACK: {feedback}", flush=True)

                        pbar.set_postfix({"feedbacks": len(predictions)})
                    last_feedback_elapsed = elapsed_time
                except torch.cuda.OutOfMemoryError:
                    oom_count += 1
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    while len(coach.feature_buffer) > 20:
                        coach.feature_buffer.popleft()
                except Exception as e:
                    print(f"\nFeedback error at frame {frame_count}: {e}", flush=True)


    cap.release()
    processing_time = time.time() - start_time

    print("\n" + "="*60)
    print(f"Processed {frame_count} frames in {processing_time:.1f}s")
    print(f"Generated {len(predictions)} feedback messages")
    if oom_count > 0:
        print(f"GPU OOM events: {oom_count}")

    if predictions:
        print("\nFeedback timeline:")
        for i, (ts, fb) in enumerate(zip(pred_timestamps, predictions), 1):
            print(f"{i:02d}. [{ts:6.1f}s] {fb}")

    print("="*60)


Starting FitCoach Live Feedback...

Video: SquatsTrimmed2.mp4
Exercise: squats
Feedback interval: 15.0s
Feature frequency: 2 fps (every 0.50s)


Video info: 2277 frames, 30.00 fps, 75.9s


Processing:  15%|█▌        | 348/2277 [00:00<00:00, 3474.48it/s]


[  15.0s] FEEDBACK: Let's squat!


Processing:  31%|███       | 696/2277 [00:02<00:06, 243.72it/s, feedbacks=1] 


[  30.0s] FEEDBACK: Let's squat!


...


[  60.0s] FEEDBACK: Let's get squatting!


Processing:  86%|████████▌ | 1951/2277 [00:09<00:01, 178.65it/s, feedbacks=4]


[  75.0s] FEEDBACK: Let's squat!


Processing: 100%|█████████▉| 2271/2277 [00:12<00:00, 187.79it/s, feedbacks=5]


Processed 2271 frames in 12.1s
Generated 5 feedback messages

Feedback timeline:
01. [  15.0s] Let's squat!
02. [  30.0s] Let's squat!
03. [  45.0s] Let's squat!
04. [  60.0s] Let's get squatting!
05. [  75.0s] Let's squat!
